In [6]:
# Install necessary libraries
# -q quiets the output logs
!pip install transformers datasets accelerate gradio torch -q

import torch
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset

# Check device (Use GPU if available for faster training)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device found: {device}")

Device found: cuda


In [7]:
# 1. Create dummy data (mimicking your 500+ quotes dataset)
# We duplicate a small list to simulate a larger dataset for the training loop
data_samples = [
    {"text": "Success is walking from failure to failure with no loss of enthusiasm."},
    {"text": "The only limit to our realization of tomorrow will be our doubts of today."},
    {"text": "Do what you can, with what you have, where you are."},
    {"text": "It always seems impossible until it is done."},
    {"text": "Keep your face always toward the sunshine—and shadows will fall behind you."},
    {"text": "The future belongs to those who believe in the beauty of their dreams."},
    {"text": "You are never too old to set another goal or to dream a new dream."},
    {"text": "You do not find the happy life. You make it."},
    {"text": "In the middle of every difficulty lies opportunity."},
    {"text": "Start where you are. Use what you have. Do what you can."},
    {"text": "Happiness is not something ready made. It comes from your own actions."},
    {"text": "Whatever you are, be a good one."},
    {"text": "If you cannot do great things, do small things in a great way."},
    {"text": "Opportunities don't happen, you create them."},
    {"text": "It is never too late to be what you might have been."}
] * 50  # We multiply by 50 to give the model enough practice iterations

# 2. Convert to Hugging Face Dataset object
dataset = Dataset.from_list(data_samples)

# 3. Split into Train and Test (optional but good practice)
dataset = dataset.train_test_split(test_size=0.1)
print("Dataset prepared:", dataset)

Dataset prepared: DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 675
    })
    test: Dataset({
        features: ['text'],
        num_rows: 75
    })
})


In [8]:
model_checkpoint = "distilgpt2"  # 82M Parameter model (GPT-2 Mini)

# 1. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# CRITICAL FIX: GPT-2 has no pad token by default. We set it to EOS.
tokenizer.pad_token = tokenizer.eos_token

# 2. Load Model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
model.config.pad_token_id = model.config.eos_token_id  # Update model config

# 3. Tokenization Function
def tokenize_function(examples):
    # Truncate to 128 tokens for speed
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

In [9]:
# 1. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./gpt2-mini-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,             # 3 passes over the data
    per_device_train_batch_size=8,  # Process 8 samples at a time
    learning_rate=5e-5,             # Standard LR for fine-tuning
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="no",             # Don't save checkpoints (saves disk space for demo)
    report_to="none"                # Disable external logging (like WandB)
)

# 2. Data Collator (Handles batching)
# mlm=False means we are doing Causal Language Modeling (Predict next word), not Masked LM
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 3. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

# 4. Train!
print("Starting training...")
trainer.train()
print("Training complete.")

# 5. Save the model locally so we can reload it for the app
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

Starting training...


Step,Training Loss
10,3.150200
20,1.940500
30,0.965300
40,0.585100
50,0.476400
60,0.327100
70,0.310000
80,0.232900
90,0.183700
100,0.167000


Training complete.


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [10]:
# Reload the fine-tuned model for inference
fine_tuned_model = AutoModelForCausalLM.from_pretrained("./final_model").to(device)
fine_tuned_tokenizer = AutoTokenizer.from_pretrained("./final_model")

def generate_text(prompt, temperature, top_k, top_p):
    # 1. Encode the prompt
    inputs = fine_tuned_tokenizer(prompt, return_tensors="pt").to(device)

    # 2. Generate output with decoding strategies
    with torch.no_grad():
        outputs = fine_tuned_model.generate(
            inputs.input_ids,
            max_length=40,            # Max length of generated text
            temperature=temperature,   # Creativity
            top_k=int(top_k),          # Limit to top K words
            top_p=top_p,               # Nucleus sampling
            do_sample=True,            # Required for creative generation
            pad_token_id=fine_tuned_tokenizer.eos_token_id,
            repetition_penalty=1.2     # Reduce repetition
        )

    # 3. Decode back to text
    return fine_tuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Define the UI
interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Enter Prompt", placeholder="Success requires..."),
        gr.Slider(0.1, 2.0, value=0.5, label="Temperature (Creativity)"),
        gr.Slider(1, 100, value=50, step=1, label="Top-K"),
        gr.Slider(0.1, 1.0, value=0.9, label="Top-P")
    ],
    outputs=gr.Textbox(label="Generated Output"),
    title="GPT-2 Mini (82M) Text Generator",
    description="A fine-tuned model for generating motivational text with controllable parameters."
)

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8e51d17c365dd82321.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
